## 46단계: Optimizer로 수행하는 매개변수 갱신

> .

### 46.1 Optimizer 클래스

Optimizer: 매개변수 갱신을 위한 기반 클래스

In [5]:
# dezero/optimizers.py

class Optimizer:
    def __init__(self):
        self.target = None
        self.hooks = []
        
    def setup(self, target):
        self.target = target
        return self

    def update(self):
        # None 이외의 매개변수를 리스트에 모아둠
        params = [p for p in self.target.params() if p.grad is not None]
        
        # 전처리(옵션)
        for f in self.hooks:
            f(params)
            
        # 매개변수 갱신
        for param in params:
            self.update_one(param)
        
    def update_one(self, param):
        raise NotImplementedError()
    
    def add_hook(self, f):
        self.hooks.append(f)

### 46.2 SGD 클래스 구현

In [6]:
# dezero/optimizers.py

class SGD(Optimizer):
    def __init__(self, lr=0.01):
        super().__init__()
        self.lr = lr

    def update_ont(self, param):
        param.data -= self.lr * param.grad.data

### 46.3 SGD 클래스를 사용한 문제 해결

In [7]:
import numpy as np
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP


np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

lr = 0.2
max_iter = 10000
hidden_size = 10

model = MLP((hidden_size, 1))
optimizer = optimizers.SGD(lr).setup(model)

for i in range(max_iter):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward()

    optimizer.update()
    if i % 1000 == 0:
        print(loss)

Variable(0.8165178479901415)
Variable(0.2499028014603371)
Variable(0.24609874026436834)
Variable(0.23721586110833612)
Variable(0.20793217994822147)
Variable(0.12311919860580518)
Variable(0.07888168390348675)
Variable(0.07656075297857305)
Variable(0.07643364647799142)
Variable(0.0761937449484299)


### 46.4 SGD 이외의 최적화 기법

Momentum 구현

<img src="images/식 46.1.png" width=400/>
<br/>
<img src="images/식 46.2.png" width=380/>

In [8]:
# dezero/optimizers.py

class MomentumSGD(Optimizer):
    def __init__(self, lr=0.01, momentum=0.9):
        super().__init__()
        self.lr = lr
        self.momentum = momentum
        self.vs = {}

    def update_one(self, param):
        v_key = id(param)
        if v_key not in self.vs:
            self.vs[v_key] = np.zeros_like(param.data)

        v = self.vs[v_key]
        v *= self.momentum
        v -= self.lr * param.grad.data
        param.data += v